In [1]:
import sys
sys.path.append('..')
from omegaconf import OmegaConf, DictConfig
from steer.vector_generators.vector_generators import BaseVectorGenerator
from steer.datasets import prepare_train_dataset
from steer.vector_appliers.vector_applier import BaseVectorApplier
from steer.datasets import prepare_generation_datasets

top_cfg = OmegaConf.load("./config_loreft_translate.yaml")
# top_cfg.model_name_or_path = model_path
# top_cfg.device = "cuda:0"
top_cfg

nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


{'model_name_or_path': 'C:/git/axbench-main/google/gemma-2-2b-it/', 'torch_dtype': 'bfloat16', 'device': 'cuda', 'use_chat_template': True, 'system_prompt': '', 'steer_train_hparam_paths': ['../hparams/Steer/loreft_hparams/generate_loreft.yaml'], 'steer_train_dataset': ['translate'], 'steer_vector_output_dirs': 'vectors/gemma-2-2b-it/', 'apply_steer_hparam_paths': ['../hparams/Steer/loreft_hparams/apply_loreft.yaml'], 'steer_vector_load_dir': ['vectors/gemma-2-2b-it/translate/loreft_vector'], 'generation_data': ['nontoxic'], 'generation_data_size': 5, 'generation_output_dir': 'vectors/gemma-2-2b-it/translate_loreft_results/', 'num_responses': 1, 'steer_from_end_position': False, 'generate_orig_output': True, 'generation_params': {'max_new_tokens': 100, 'temperature': 0.9}}

### Generate Steering Vector

In [2]:
datasets = {
    'translate':[
        {'question': 'What is the capital of France?', 
        'matching':'法国的首都是巴黎。', 
        'not_matching':'The capital of France is Paris.'}, 
        {'question': '世界上最长的河流是什么？', 
        'matching': '世界上最长的河流是尼罗河。', 
        'not_matching':'The longest river in the world is the Nile River.'}
    ]
}

# Or use the datasets from config.yaml
# datasets = prepare_train_dataset(top_cfg)

vector_generator = BaseVectorGenerator(top_cfg)
vector_generator.generate_vectors(datasets)

Loading loreft hparams from ../hparams/Steer/loreft_hparams/generate_loreft.yaml ...
LOREFT_0 Generator Hyperparameters:
LoReFTHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', seed=42, model_name_or_path='C:/git/axbench-main/google/gemma-2-2b-it/', device='cuda', use_cache=True, generate_orig_output=True, alg_name='loreft', steer_vector_output_dir='vectors/gemma-2-2b-it/', steer_train_dataset=['translate'], reft_layers=[5, 10, 15, 20], lr=0.0009, n_epochs=24, max_length=512, batch_size=1, gradient_accumulation_steps=2, subset=None, low_rank_dimension=4, position='f5+l5', weight_decay=0.0, save_vectors=True)
Generating loreft_0 vectors ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

trainable intervention params: 73,744 || trainable model params: 0
model params: 2,614,341,888 || trainable%: 0.002820748133153119


lr 0.000037 || loss 0.000269 : 100%|██████████| 24/24 [00:07<00:00,  3.24it/s]

Saving vectors to vectors/gemma-2-2b-it/translate\loreft_vector ...


{'translate': {'loreft_0': None}}

### Apply Steering Vector

In [3]:
vector_applier = BaseVectorApplier(top_cfg)
vector_applier.apply_vectors()

datasets = {f'translate_test':[{'input':'你好'},{'input':'how are you'},{'input':'hello'}]}
vector_applier.generate(datasets)
vector_applier.reset_loreft()

LOREFT Applier Hyperparameters:
ApplyLoReFTHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', seed=42, model_name_or_path='C:/git/axbench-main/google/gemma-2-2b-it/', device='cuda', use_cache=True, generate_orig_output=True, alg_name='loreft', steer_vector_load_dir='vectors/gemma-2-2b-it/translate/loreft_vector', reft_layers=[5, 10, 15, 20], max_length=512, batch_size=1, low_rank_dimension=4, position='f5+l5', temperature=1.0)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Applying loreft vectors or prompt to model successfully !

Saving results to vectors/gemma-2-2b-it/translate_loreft_results/translate_test_results.json

===== translate_test Results =====

----- Input -----
你好

----- Orig Output-----
['你好！ 👋  \n\n很高兴认识你！ 😊  有什么我能帮你的吗？ \n']

----- Steered Output-----
['你好！我很乐意 assisting you!  请说你想知道什么。 😊 \n']

